# Rpi serial ports

* [Official Documentation](https://www.raspberrypi.org/documentation/configuration/uart.md)
* [
Pi-4 Activating additional UART ports](https://www.raspberrypi.org/forums/viewtopic.php?t=244827)

* default:
   * UART0 (PL011): 
       * secondary (Bluetooth)
       * /dev/serial0
   * UART1 (mini UART): 
       * primary @ GPIO 14 & 15
       * /dev/serial1
   * UART2 ... 5 (PL011)

```
        TXD RXD CTS RTS     MOTOR_HAT
uart0   14  15              RX, TX (STM32 UART 3, DFU)
uart1   14  15              
uart2   0   1   2   3       
uart3   4   5   6   7       RX_AUX, TX_AUX (STM32 UART 6, no DFU)
uart4   8   9   10  11      
uart5   12  13  14  15      
```

## Enable UARTS

Add the following to the `[p4]` section in `/boot/config.txt`:

```
[pi4]
dtoverlay=uart3
```

#### `enable_uart=1`

All enable_uart=1 does is restore the default serial UART to pins 8 & 10 (GPIO 14 & 15), which is assigned to the Bluetooth module on Pi models with wireless (4B, Pi0W and 3B/3B+).

**Messes up BT, leave alone (and these pins)**

In [13]:
!tail /boot/config.txt 

# Enable audio (loads snd_bcm2835)
dtparam=audio=on

[pi4]
# Enable DRM VC4 V3D driver on top of the dispmanx display stack
dtoverlay=vc4-fkms-v3d
max_framebuffers=2

[all]
#dtoverlay=vc4-fkms-v3d


## Port functions

In [16]:
!raspi-gpio get 14-15

GPIO 14: level=1 fsel=0 func=INPUT pull=NONE
GPIO 15: level=1 fsel=0 func=INPUT pull=UP


---
**Bluetooth UART selection**

---

* Bluetooth apparently is on GPIO 32/33
    * UART0 and UART1 (mini UART) are available on these pins
    * By default uses UART0, can be switched to mini UART (?)
* Only UART0 & UART1 are available on GPIO 14/15!
* Hence GPIO 14/15 can only use the MiniUART
* Use different pins for STM32!!!
    * E.g. UART5 on GPIO 12/13
    * requires board mod (incompatible with Pi3 & PiZero)
    
**Compromise?**

* Flash via USB (dfutil)
* Use mini UART (UART3 on STM32) for REPL
* Use UART3 (Pi, UART6 on STM32) for comm

Adding `enable_uart=1` to config.txt on a Pi4 enables ttyS0 (UART1 = mini UART) on GPIOs 14 & 15 (Alt5), leaving UART0 driving the Bluetooth interface on 30-33 (Alt3). 

Adding `dtoverlay=disable-bt` switches the UART roles so that UART0 is mapped to 14 & 15 (Alt0), leaving UART1 unmapped.

In [15]:
!raspi-gpio get 32-41 

GPIO 32: level=1 fsel=7 alt=3 func=TXD0 pull=NONE
GPIO 33: level=1 fsel=7 alt=3 func=RXD0 pull=UP
GPIO 34: level=1 fsel=7 alt=3 func=SD1_CLK pull=NONE
GPIO 35: level=1 fsel=7 alt=3 func=SD1_CMD pull=UP
GPIO 36: level=1 fsel=7 alt=3 func=SD1_DAT0 pull=UP
GPIO 37: level=1 fsel=7 alt=3 func=SD1_DAT1 pull=UP
GPIO 38: level=1 fsel=7 alt=3 func=SD1_DAT2 pull=UP
GPIO 39: level=1 fsel=7 alt=3 func=SD1_DAT3 pull=UP
GPIO 40: level=0 fsel=4 alt=0 func=PWM1_0 pull=NONE
GPIO 41: level=0 fsel=4 alt=0 func=PWM1_1 pull=NONE


In [21]:
!raspi-gpio funcs | grep XD2

0, UP, SDA0, SA5, PCLK, SPI3_CE0_N, TXD2, SDA6
1, UP, SCL0, SA4, DE, SPI3_MISO, RXD2, SCL6


In [11]:
!dtoverlay -h uart0 | grep GPIO
!dtoverlay -h uart1 | grep GPIO
!dtoverlay -h uart3 | grep GPIO

Params: txd0_pin                GPIO pin for TXD0 (14, 32 or 36 - default 14)
        rxd0_pin                GPIO pin for RXD0 (15, 33 or 37 - default 15)
Params: txd1_pin                GPIO pin for TXD1 (14, 32 or 40 - default 14)
        rxd1_pin                GPIO pin for RXD1 (15, 33 or 41 - default 15)
Info:   Enable uart 3 on GPIOs 4-7. BCM2711 only.
Params: ctsrts                  Enable CTS/RTS on GPIOs 6-7 (default off)


In [12]:
!dtoverlay -h uart3 | grep GPIO

Info:   Enable uart 3 on GPIOs 4-7. BCM2711 only.
Params: ctsrts                  Enable CTS/RTS on GPIOs 6-7 (default off)


## 1) Remap Serial Ports (BT uses miniuart, Pi header uses main uart)

Note: also fix Pi clock frequency for BT to work (havn't done that).

In [7]:
!tail /boot/config.txt

# Enable audio (loads snd_bcm2835)
dtparam=audio=on

[pi4]
# Enable DRM VC4 V3D driver on top of the dispmanx display stack
dtoverlay=vc4-fkms-v3d
max_framebuffers=2

[all]
#dtoverlay=vc4-fkms-v3d


In [2]:
!ls /dev/serial*

/dev/serial1


Pi 3, Pi Zero W: serial0->ttyS0 points to GPIO pins 8, 10.

So where possible refer to the serial port via it’s alias of “serial0” and your code should work on both Raspberry Pi 3 and other Raspberry Pi’s.

In [3]:
!ls -l /dev/serial* /dev/ttyA*

lrwxrwxrwx 1 root root          7 Apr 20 17:10 /dev/serial1 -> ttyAMA0
crw-rw---- 1 root dialout 204, 64 Apr 20 17:10 /dev/ttyAMA0
crw-rw---- 1 root dialout 204, 65 Apr 20 17:10 /dev/ttyAMA1
crw-rw---- 1 root dialout 204, 66 Apr 20 17:10 /dev/ttyAMA2


In [3]:
%connect '/dev/ttyAMA2'

Device not available: '/dev/ttyAMA2'


## 2) Disable the Console

By default the Raspberry Pi uses the serial port for the “console” login and via a software service called “getty”.
If you are using the serial port for anything other than the console you need to disable it. This will be slightly different depending on whether you are running a Raspberry Pi 3 or not.
For non Raspberry Pi 3 machines, remember it’s /dev/ttyAMA0 that is linked to the getty (console) service.

For Raspberry Pi 3’s the command is referencing /dev/ttyS0:

In [3]:
!sudo systemctl stop serial-getty@ttyS0.service
!sudo systemctl disable serial-getty@ttyS0.service

In [3]:
# remove 'console=serial0,115200' from `/boot/cmdline.txt`

%cat  /boot/cmdline.txt

console=tty1 root=PARTUUID=f0071ff7-02 rootfstype=ext4 elevator=deadline fsck.repair=yes rootwait


## 3) Loopback Test

Disable uart0 if its pins (8,10) are shorted to uart4 (pins 21, 24) ...

```
[pi4]
dtoverlay=uart3
dtoverlay=uart4
enable_uart=0
```

short RX and TX pins, then:

In [3]:
%%host


import serial, time

port = "/dev/serial0"
port = "/dev/ttyAMA1"  # uart3
# port = "/dev/ttyAMA2"  # uart4


with serial.Serial(port, baudrate=9600, timeout=3.0) as dev:
    for i in range(10):
        dev.write(b"Hello world!")
        rcv = dev.read(12)
        print(rcv)

b'Hello world!'
b'Hello world!'
b'Hello world!'
b'Hello world!'
b'Hello world!'
b'Hello world!'
b'Hello world!'
b'Hello world!'
b'Hello world!'
b'Hello world!'


In [2]:
!ls /dev/ttyA*

/dev/ttyAMA0  /dev/ttyAMA1  /dev/ttyAMA2


In [13]:
!tail /boot/config.txt

# Enable DRM VC4 V3D driver on top of the dispmanx display stack
dtoverlay=vc4-fkms-v3d
max_framebuffers=2
dtoverlay=uart3
dtoverlay=uart4

[all]
#dtoverlay=vc4-fkms-v3d
gpu_mem=32
enable_uart=0
